In [277]:
import sys 
import os
from pathlib import Path
import csv 
import requests
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
import pylab as pl
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import akshare as ak
import sqlite3 as sql
import talib as ta
from tqdm import tqdm
from itertools import combinations
from sklearn.preprocessing import OrdinalEncoder
from datetime import datetime,timedelta
plt.rcParams['font.sans-serif'] = ['SimHei']
pd.set_option('display.unicode.east_asian_width',True)
plt.rcParams['axes.unicode_minus']=False
mpl.use('TkAgg')

In [278]:
all_A_listed = ak.stock_zh_a_spot_em()
conn = sql.connect('symbol.db')
all_A_symbols = all_A_listed[['代码', '名称']].sort_values('代码')
# all_A_symbols.to_sql(name='symbols', con=conn, if_exists='append')
all_A_listed=all_A_listed.sort_values('代码')
#股票

sse=ak.stock_sh_a_spot_em()
sse
szse=ak.stock_sz_a_spot_em()
szse 
bse=ak.stock_bj_a_spot_em()
bse
ex = input('exchange:(可输入sse，szse和bse，其他输入默认为所有交易所股票)')
if ex =='sse':
    stock_ex=sse
elif ex=='szse':
    stock_ex=szse
elif ex=='bse':
    stock_ex=bse
else:
    stock_ex=all_A_listed

com_data=all_A_listed['代码'].tolist()

stock_ex=stock_ex.dropna()
stock_ex

exchange:(可输入sse，szse和bse，其他输入默认为所有交易所股票) 


,序号,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,振幅,最高,...,量比,换手率,市盈率-动态,市净率,总市值,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
838,839,000001,平安银行,9.46,-0.42,-0.04,1357217.0,1.286857e+09,1.89,9.56,...,0.67,0.70,3.47,0.46,1.835800e+11,1.835765e+11,-0.11,-0.11,-10.75,0.75
1277,1278,000002,万 科Ａ,9.60,-1.94,-0.19,674789.0,6.527274e+08,2.96,9.85,...,0.57,0.69,6.31,0.45,1.145348e+11,9.328001e+10,-0.10,-0.31,-19.53,-8.22
5451,5452,000004,国华网安,12.35,-9.99,-1.37,70312.0,9.050359e+07,10.71,13.82,...,0.73,5.57,-34.83,5.15,1.634896e+09,1.559658e+09,0.00,0.00,-26.84,-23.43
3004,3005,000005,ST星源,0.94,-5.05,-0.05,163193.0,1.572986e+07,7.07,1.01,...,1.22,1.54,-89.13,0.83,9.950246e+08,9.944699e+08,0.00,0.00,-33.33,-12.15
4693,4694,000006,深振业Ａ,4.17,-7.13,-0.32,490279.0,2.094531e+08,6.01,4.43,...,0.72,3.63,-3444.61,0.72,5.629479e+09,5.629447e+09,-0.48,-0.24,-6.92,-9.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4563,4564,873693,阿为特,33.34,-6.95,-2.49,17699.0,6.171719e+07,10.13,36.73,...,0.56,8.72,89.68,7.57,2.423818e+09,6.768020e+08,0.00,-0.95,16.37,-19.74
1045,1046,873703,广厦环能,30.33,-1.21,-0.37,22023.0,6.455209e+07,8.63,30.66,...,1.74,10.50,13.69,2.51,2.332377e+09,6.363234e+08,-0.16,0.00,29.34,-21.18
90,91,873726,卓兆点胶,31.63,4.32,1.31,14615.0,4.605875e+07,10.85,32.97,...,1.00,8.38,29.31,4.32,2.596103e+09,5.516720e+08,-0.25,-0.66,44.03,-27.52
3246,3247,873806,云星宇,11.55,-5.33,-0.65,257265.0,3.088784e+08,11.23,12.78,...,0.91,33.88,29.46,2.51,3.347960e+09,8.771461e+08,-0.35,0.61,149.46,149.46


In [279]:
stock_code=str(input('stock code:'))
#通过改变timedelta系数来确定获取几日数据（以一年为例）
today=datetime.today()
start_date = today - timedelta(days=365)
today = today.strftime('%Y%m%d')
start_date = start_date.strftime('%Y%m%d')

while True:
    if stock_code in com_data:
        
        stock_data_spe=ak.stock_zh_a_hist(symbol=stock_code, period="daily", start_date=start_date, end_date=today)
    
        com_name=ak.stock_individual_info_em(symbol=stock_code).loc[5,'value']

        break
    else:
        print('wrong stock code,input a correct code again:')
        stock_code=input()

stock_data_spe.fillna(0,inplace=True)
print('股票名称：',com_name)
stock=all_A_listed[all_A_listed['代码']==stock_code]
print(stock)
stock_data_spe['市盈率']=stock['市盈率-动态'].tolist()[0]
stock_data_spe['市净率']=stock['市净率'].tolist()[0]
stock_data_spe['流通市值']=stock['流通市值'].tolist()[0]
stock_data_spe['日期']=pd.to_datetime(stock_data_spe['日期'])
stock_data_spe['量比']=stock['量比'].tolist()[0]

stock_data_spe

stock code: 600225


股票名称： 卓朗科技
      序号    代码      名称  最新价  涨跌幅  涨跌额    成交量      成交额  \
2725  2726  600225  卓朗科技    3.01   -4.75   -0.15  221355.0  68018418.0   

      振幅  最高  ...  量比  换手率  市盈率-动态  市净率        总市值  \
2725   5.7  3.18  ...  1.15    0.65       156.38     4.5  1.026830e+10   

          流通市值  涨速  5分钟涨跌  60日涨跌幅  年初至今涨跌幅  
2725  1.026056e+10 -0.33      -0.33       -31.9          -20.79  

[1 rows x 23 columns]


,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,市盈率,市净率,流通市值,量比
0,2023-01-31,7.48,7.28,7.68,7.21,629356,467507644.0,6.27,-2.93,-0.22,1.85,156.38,4.5,1.026056e+10,1.15
1,2023-02-01,7.29,7.32,7.38,7.17,422669,306989614.0,2.88,0.55,0.04,1.24,156.38,4.5,1.026056e+10,1.15
2,2023-02-02,7.27,7.19,7.35,7.16,322844,233401684.0,2.60,-1.78,-0.13,0.95,156.38,4.5,1.026056e+10,1.15
3,2023-02-03,7.21,7.46,7.46,7.11,411456,301519143.0,4.87,3.76,0.27,1.21,156.38,4.5,1.026056e+10,1.15
4,2023-02-06,7.58,7.45,7.58,7.29,378630,282046276.0,3.89,-0.13,-0.01,1.11,156.38,4.5,1.026056e+10,1.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2024-01-25,3.23,3.37,3.37,3.18,212852,69956902.0,5.90,4.66,0.15,0.62,156.38,4.5,1.026056e+10,1.15
245,2024-01-26,3.37,3.38,3.44,3.31,221181,74978575.0,3.86,0.30,0.01,0.65,156.38,4.5,1.026056e+10,1.15
246,2024-01-29,3.41,3.26,3.41,3.25,150903,49976576.0,4.73,-3.55,-0.12,0.44,156.38,4.5,1.026056e+10,1.15
247,2024-01-30,3.21,3.16,3.29,3.14,159466,51423946.0,4.60,-3.07,-0.10,0.47,156.38,4.5,1.026056e+10,1.15


In [292]:
# EMA(Exponential Moving Average), 指数移动平均线
num_periods_fast = 12  # 快速EMA的时间周期，12
# K:平滑常数，常取2/(n+1)
K_fast = 2 / (num_periods_fast + 1)  # 快速EMA平滑常数
ema_fast = 0
num_periods_slow = 40  # 慢速EMA的时间周期，26
K_slow = 2 / (num_periods_slow + 1)  # 慢速EMA平滑常数
ema_slow = 0
num_periods_macd = 9  # MACD EMA的时间周期，9
K_macd = 2 / (num_periods_macd + 1)  # MACD EMA平滑常数
ema_macd = 0

ema_fast_values = []  
ema_slow_values = []  
macd_values = []  
macd_signal_values = []  
# MACD - MACD-EMA
MACD_hist_values = []  
for close_price in stock_data_spe['收盘']:
    if ema_fast == 0:  # 第一个值
        ema_fast = close_price
        ema_slow = close_price
    else:
        ema_fast = (close_price - ema_fast) * K_fast + ema_fast
        ema_slow = (close_price - ema_slow) * K_slow + ema_slow

    ema_fast_values.append(ema_fast)
    ema_slow_values.append(ema_slow)

	# MACD is fast_MA - slow_EMA
    macd = ema_fast - ema_slow  
    if ema_macd == 0:
        ema_macd = macd
    else:
    	# signal is EMA of MACD values
        ema_macd = (macd - ema_macd) * K_macd + ema_macd  
    macd_values.append(macd)
    macd_signal_values.append(ema_macd)
    MACD_hist_values.append(macd - ema_macd)

stock_data_spe = stock_data_spe.assign(ClosePrice=pd.Series(stock_data_spe['收盘'], index=stock_data_spe.index))
stock_data_spe = stock_data_spe.assign(FE12=pd.Series(ema_fast_values, index=stock_data_spe.index))
stock_data_spe = stock_data_spe.assign(SE26=pd.Series(ema_slow_values, index=stock_data_spe.index))
stock_data_spe = stock_data_spe.assign(MACD=pd.Series(macd_values, index=stock_data_spe.index))
stock_data_spe = stock_data_spe.assign(EMA_MACD=pd.Series(macd_signal_values, index=stock_data_spe.index))
stock_data_spe = stock_data_spe.assign(MACDFS=pd.Series(MACD_hist_values, index=stock_data_spe.index))

close_price = stock_data_spe['ClosePrice']
ema_f = stock_data_spe['FE12']
ema_s = stock_data_spe['SE26']
macd = stock_data_spe['MACD']
ema_macd = stock_data_spe['EMA_MACD']
MACDFS = stock_data_spe['MACDFS']

fig = plt.figure(figsize=(16, 10))
gs = plt.GridSpec(2, 1, height_ratios=[1, 1], hspace=0.1)  # 设置子图网格，height_ratios用于调整子图高度比例，hspace用于调整子图之间的垂直间距

# 第一个子图
ax0 = plt.subplot(gs[0])
ax0.set_ylabel('close price')
ax0.set_title(f'{com_name}(股票代码{stock_code})股价及MACD')
close_price.plot(ax=ax0, color='#000000', lw=1., legend=True, use_index=False)
ema_f.plot(ax=ax0, color='#00CCFF', lw=1., legend=True, use_index=False)
ema_s.plot(ax=ax0, color='#FF3366', lw=1., legend=True, use_index=False)
ax0.xaxis.set_major_locator(mdates.YearLocator())  
ax0.xaxis.set_minor_locator(mdates.DayLocator(interval=30))  # 设置主要刻度为每周，可以根据需要调整



# 第二个子图
ax1 = plt.subplot(gs[1], sharex=ax0)
macd.plot(ax=ax1, color='#FFCC00', lw=1., legend=True, use_index=False)
ema_macd.plot(ax=ax1, color='#000000', lw=1., legend=True, use_index=False)

colors = np.where(stock_data_spe['MACDFS'] >= 0, '#FF3333', '#33FF33')
stock_data_spe['MACDFS'].plot(ax=ax1, color=colors, kind='bar', legend=True, sharex=ax0)

ax1.xaxis.set_major_locator(mdates.YearLocator())  
ax1.xaxis.set_minor_locator(mdates.DayLocator(interval=5))  # 设置主要刻度为每周，可以根据需要调整


# 使用自定义日期格式
date_format = mdates.DateFormatter('%Y-%m-%d')  # 修改日期格式
ax1.xaxis.set_minor_formatter(date_format)  # 设置次要刻度的日期格式为年月日


# 设置X轴标签
plt.xlabel('时间（一年基准）')

plt.xlim(xmin=1)

plt.show()

Locator attempting to generate 3953 ticks ([-9.0, ..., 19751.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 3953 ticks ([-9.0, ..., 19751.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 3953 ticks ([-9.0, ..., 19751.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 3953 ticks ([-9.0, ..., 19751.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 3953 ticks ([-9.0, ..., 19751.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 3953 ticks ([-9.0, ..., 19751.0]), which exceeds Locator.MAXTICKS (1000).
